In [ ]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
from brian2 import *


## Preprocessing files

def preprocess_audio(file_path):
    y, sr = librosa.load(file_path)
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    chromagram = librosa.feature.chroma_stft(y=y, sr=sr)
    
    # Stacking features horizontally
    combined_features = np.vstack([mfccs, spectral_contrast, chromagram])
    # Encoding step: Normalize to range [0, 1]
    encoded_features = (combined_features - np.min(combined_features)) / (np.max(combined_features) - np.min(combined_features))
    
    return encoded_features


def count_files(directory):
    return sum([len(files) for _, _, files in os.walk(directory)])

def load_and_preprocess_data(directory):
    data = []
    labels = []

    total_files = count_files(directory)
    processed_files = 0

    for label, time_sig in enumerate(['1_4', '2_4', '3_4', '4_4']):
        for file in os.listdir(f'{directory}/{time_sig}'):
            file_path = os.path.join(directory, time_sig, file)
            processed_data = preprocess_audio(file_path)
            data.append(processed_data)
            labels.append(label)

            # Update progress
            processed_files += 1
            print(f"Processed {processed_files}/{total_files} files ({(processed_files/total_files)*100:.2f}%).", end='\r')

    return data, labels

print("Loading and preprocessing training data...")
training_data, training_labels = load_and_preprocess_data('training_data')
print("\nLoading and preprocessing validation data...")
validation_data, validation_labels = load_and_preprocess_data('validation_data')
print("\nDone with preprocessing!")